In [1]:
%pip install mcts-simple

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ast
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from mcts_simple import UCT, Game

# root_dir = "/kaggle/input/santa-2023"
root_dir = "competition_data"

/venv-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
puzzles_df = pd.read_csv(os.path.join(root_dir, 'puzzles.csv'))
puzzles_df.head()

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0


In [4]:
puzzle_info_df = pd.read_csv(os.path.join(root_dir, 'puzzle_info.csv'))
puzzle_info_df

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


### Some quick notes about puzzle structure
- Cube_x/x/x puzzles have a set of moves {f0, f1, ..., f(x-1), r0, r1, ..., r(x-1), d0, d1, ..., d(x-1)}
- Wreath_x/x puzzles have a set of moves {l, r}
- Globe_x/y puzzles have a set of moves {r0, r1, ..., r(x), f0, f1, ..., f(2y-1)}

In fact, Cube_x/x/x puzzles are just Rubik's Cube type puzzles, Wreath_x/x puzzles are Hungarian Rings (or Devil's Circles), and Globe_x/y puzzles are Rainbow Masterball (or just Masterball) puzzles. Reference: https://www.permutationpuzzles.org/rubik/webnotes/rubik.pdf

This helps us understand the moves as well: The cube moves are just rotations of an x-by-x-by-1 layer of cubes, the wreath moves are just a counterclockwise rotation of either the left or right ring, and the globe moves are either a rotation of a latitude-wise layer (of which there are x+1) or a flipping along a line of longitude (of which there are 2y). 

Of course, negative moves are also possible, though it would not make sense in the move sequence to have, e.g., `f0` followed by `-f0` so we should restrict that case if possible.

In [5]:
submission_df = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
submission_df

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


The submission data frame provides a baseline solution for all of the puzzles, though we expect to be able to improve upon many of them.

In [6]:
# Count the total number of moves in the submission from each puzzle type
submission_df['total_moves'] = submission_df['moves'].apply(lambda x: len(x.split(".")))
submission_df['puzzle_type'] = puzzles_df.loc[submission_df['id']]['puzzle_type'].values
submission_df['num_puzzles'] = 1
submission_group_df = submission_df.groupby('puzzle_type').agg({'total_moves': 'sum', 'num_puzzles': 'sum'}).reset_index()
submission_group_df = submission_group_df.sort_values(by=['total_moves'], ascending=False)
submission_group_df

,puzzle_type,total_moves,num_puzzles
4,cube_33/33/33,372200,3
14,globe_3/33,226350,8
1,cube_19/19/19,102317,4
6,cube_5/5/5,51254,35
0,cube_10/10/10,50465,5
19,globe_8/25,47513,2
3,cube_3/3/3,36661,120
5,cube_4/4/4,35320,60
10,cube_9/9/9,34550,5
15,globe_3/4,29526,15


We can see from the above analysis that the largest contributions to the overall score are first from the 33x33x33 Rubik's cube, then the 3x33 Masterball, then the 19x19x19 Rubik's cube. The Hungarian Rings puzzles are worth about 100,000 moves combined out of 1.2 million, so we can set those aside if we want a direct route to improving our score.

In [11]:
# Sort submission by total moves
submission_df = submission_df.sort_values(by=['total_moves'], ascending=False)
submission_df.iloc[250:300]

,id,moves,total_moves,puzzle_type,num_puzzles
292,292,-r.l.l.r.-l.-l.-r.-l.r.l.-r.l.l.-r.-l.-r.-l.-r...,378,wreath_6/6,1
137,137,r2.r0.f2.-r1.-f0.-f1.-f2.f1.d0.-f2.r2.r1.d1.r1...,376,cube_3/3/3,1
80,80,d2.f0.-d0.r0.f2.r0.r0.-d2.-f2.-f1.-f2.-r0.d0.-...,372,cube_3/3/3,1
135,135,-d1.f1.-r0.f1.r1.-f2.r2.d0.-f2.f0.r2.r0.-r1.r0...,372,cube_3/3/3,1
51,51,-r0.f0.-r2.r0.-r2.r1.r2.-f0.-f0.-r2.r1.r2.-f0....,370,cube_3/3/3,1
130,130,-f2.-f1.r0.f2.-f0.-f2.-r0.f1.f2.-f0.-f1.-r1.-r...,362,cube_3/3/3,1
119,119,r2.d1.f0.-r2.d1.d0.r1.r2.r1.f2.-f1.r0.-d2.-d0....,360,cube_3/3/3,1
146,146,-f1.-f1.-d2.f2.-r0.f1.-f2.d0.-d1.-f0.-f2.d2.f2...,358,cube_3/3/3,1
77,77,-f2.-f1.-f1.-f1.f2.-r1.-f1.-f1.f2.r2.-f0.-f2.-...,358,cube_3/3/3,1
47,47,-f0.-f2.-r2.f0.-r1.-f2.r0.f1.f2.-f0.f1.f1.-f0....,358,cube_3/3/3,1


In [24]:
# Try solving a cube puzzle with MCTS
class CubeGame(Game):
    def __init__(self, dimension: int, initial_state: str, solution_state: str, num_wildcards: int):
        self.dimension = dimension
        self.initial_state = np.array(initial_state.split(";"))
        self.solution_state = np.array(solution_state.split(";"))
        self.state = self.initial_state.copy()
        self.path = []
        self.num_wildcards = num_wildcards

        # Create the actions dictionaries
        base_actions_dict = ast.literal_eval(puzzle_info_df.loc[puzzle_info_df['puzzle_type'] == f'cube_{dimension}/{dimension}/{dimension}', 'allowed_moves'].values[-1])
        self.actions_dict = {}
        self.actions_index = {}     # Need this object to use integers to index the actions, rather than the identifying string
        for i, (k, v) in enumerate(base_actions_dict.items()):
            self.actions_dict[k] = np.array(v)
            self.actions_dict["-" + k] = np.argsort(np.array(v))
            self.actions_index[2*i] = k
            self.actions_index[2*i+1] = "-" + k
        self.actions_index_reverse = {v: k for k, v in self.actions_index.items()}
        self.actions_dict_length = len(self.actions_dict)

    def render(self) -> None:
        # Print the current state in the same format as the initial and solution states
        print(";".join(self.state))
        return
    
    def get_state(self) -> list[str]:
        return self.state

    # Single player game
    def number_of_players(self) -> int:
        return 1
    
    def current_player(self) -> int:
        return 0

    def possible_actions(self) -> list[int]:
        actions_list = list(np.arange(self.actions_dict_length))
        if len(self.path) == 0:
            return actions_list
        
        # Prevent the reverse of the action we just took from being selected right away
        last_move = self.path[-1]
        actions_list.remove(self.actions_index_reverse[last_move])
        return actions_list
    
    def take_action(self, action: int) -> None:
        # Add the action to the path
        self.path.append(self.actions_index[action])
        # Execute the permutation action
        perm = self.actions_dict[self.actions_index[action]]
        self.state = self.state[perm]
        return 

    def has_outcome(self) -> bool:
        wc_count = 0
        for i in range(len(self.state)):
            if self.state[i] != self.solution_state[i]:
                wc_count += 1

        if wc_count > self.num_wildcards:
            return False
        else:
            return True
        
    def winner(self) -> list[int]:
        return [0]

In [27]:
# Try solving a wreath puzzle with MCTS
class WreathGame(Game):
    def __init__(self, dimension: int, initial_state: str, solution_state: str, num_wildcards: int):
        self.dimension = dimension
        self.initial_state = np.array(initial_state.split(";"))
        self.solution_state = np.array(solution_state.split(";"))
        self.state = self.initial_state.copy()
        self.path = []
        self.num_wildcards = num_wildcards

        # Create the actions dictionaries
        base_actions_dict = ast.literal_eval(puzzle_info_df.loc[puzzle_info_df['puzzle_type'] == f'wreath_{dimension}/{dimension}', 'allowed_moves'].values[-1])
        self.actions_dict = {}
        self.actions_index = {}     # Need this object to use integers to index the actions, rather than the identifying string
        for i, (k, v) in enumerate(base_actions_dict.items()):
            self.actions_dict[k] = np.array(v)
            self.actions_dict["-" + k] = np.argsort(np.array(v))
            self.actions_index[2*i] = k
            self.actions_index[2*i+1] = "-" + k
        self.actions_index_reverse = {v: k for k, v in self.actions_index.items()}
        self.actions_dict_length = len(self.actions_dict)

    def render(self) -> None:
        # Print the current state in the same format as the initial and solution states
        print(";".join(self.state))
        return
    
    def get_state(self) -> list[str]:
        return self.state

    # Single player game
    def number_of_players(self) -> int:
        return 1
    
    def current_player(self) -> int:
        return 0

    def possible_actions(self) -> list[int]:
        actions_list = list(np.arange(self.actions_dict_length))
        if len(self.path) == 0:
            return actions_list
        
        # Prevent the reverse of the action we just took from being selected right away
        last_move = self.path[-1]
        actions_list.remove(self.actions_index_reverse[last_move])
        return actions_list
    
    def take_action(self, action: int) -> None:
        # Add the action to the path
        self.path.append(self.actions_index[action])
        # Execute the permutation action
        perm = self.actions_dict[self.actions_index[action]]
        self.state = self.state[perm]
        return 

    def has_outcome(self) -> bool:
        wc_count = 0
        for i in range(len(self.state)):
            if self.state[i] != self.solution_state[i]:
                wc_count += 1

        if wc_count > self.num_wildcards:
            return False
        else:
            return True
        
    def winner(self) -> list[int]:
        return [0]

In [28]:
# Test our MCTS solver with a small puzzle
# puzzle_id = 0   # 2x2x2 cube
# mcts_cube_2_solver = UCT(CubeGame(2, puzzles_df.loc[puzzle_id]['initial_state'], puzzles_df.loc[puzzle_id]['solution_state'], puzzles_df.loc[puzzle_id]['num_wildcards']), allow_transpositions=False, training=True)
# mcts_cube_2_solver.self_play(iterations=1)

puzzle_id = 292   # 6x6 wreath
mcts_wreath_6_solver = UCT(WreathGame(6, puzzles_df.loc[puzzle_id]['initial_state'], puzzles_df.loc[puzzle_id]['solution_state'], puzzles_df.loc[puzzle_id]['num_wildcards']), allow_transpositions=False, training=True)
mcts_wreath_6_solver.self_play(iterations=100)

Training: 100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


In [29]:
mcts_wreath_6_solver.training = False
mcts_wreath_6_solver.self_play()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

C;B;B;A;B;B;A;A;A;C
B;B;A;B;B;C;A;A;A;C
C;B;B;A;B;B;A;A;A;C
A;B;A;A;B;B;A;B;C;C
B;A;B;A;A;B;A;B;C;C
C;A;B;A;A;B;B;A;B;C
B;A;B;A;A;B;A;B;C;C
A;B;A;A;B;B;A;B;C;C
C;B;B;A;B;B;A;A;A;C
A;B;A;A;B;B;A;B;C;C
B;A;A;B;B;A;A;B;C;C
C;A;B;B;B;A;B;A;A;C
B;A;A;B;B;A;A;B;C;C
A;A;B;B;A;B;A;B;C;C
A;A;C;B;A;B;B;B;C;A
B;A;A;C;B;A;B;B;C;A
A;A;B;C;B;A;B;B;A;C
A;A;A;B;C;B;B;B;A;C
C;A;B;B;C;B;A;B;A;A
B;C;A;B;B;C;A;B;A;A
A;C;B;B;B;C;B;A;A;A
C;A;C;B;B;B;B;A;A;A
A;A;A;B;B;B;C;B;C;A
C;A;C;B;B;B;B;A;A;A
A;C;B;B;B;C;B;A;A;A
C;A;C;B;B;B;B;A;A;A
A;C;B;B;B;C;B;A;A;A
A;C;A;B;B;C;A;B;B;A
C;A;C;A;B;B;A;B;B;A
A;C;A;B;B;C;A;B;B;A
A;C;B;B;B;C;A;A;A;B
C;B;B;B;C;A;A;A;A;B
A;C;B;B;B;C;A;A;A;B
C;B;B;B;C;A;A;A;A;B
A;C;B;B;B;C;A;A;A;B
C;B;B;B;C;A;A;A;A;B
A;C;B;B;B;C;A;A;A;B
B;C;A;B;B;C;A;A;B;A
A;C;B;B;B;C;A;A;A;B
C;A;C;B;B;B;A;A;A;B
B;A;A;B;B;B;C;A;C;A
C;A;C;B;B;B;A;A;A;B
B;A;A;B;B;B;C;A;C;A
B;B;A;A;B;B;C;A;C;A
A;B;A;A;B;B;B;C;A;C
B;A;B;A;A;B;B;C;A;C
A;B;A;A;B;B;B;C;A;C
B;A;B;A;A;B;B;C;A;C
A;B;A;A;B;B;B;C;A;C
C;B;C;A;B;B;A;B;A;A


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]

A;B;A;A;A;B;B;C;C;B
B;A;B;A;A;A;B;C;C;B
A;B;A;A;A;B;B;C;C;B
B;B;C;A;A;B;A;B;A;C
B;B;B;C;A;A;A;B;A;C
C;B;B;C;A;A;B;A;B;A
A;C;B;B;C;A;B;A;B;A
A;C;A;B;C;A;A;B;B;B
C;A;B;C;A;A;A;B;B;B
A;A;B;C;A;A;B;B;B;C
C;A;B;C;A;A;A;B;B;B
A;A;B;C;A;A;B;B;B;C
A;B;C;A;A;A;B;B;B;C
C;B;B;A;A;A;A;B;C;B
B;B;A;A;A;C;A;B;C;B
C;B;B;A;A;A;A;B;C;B
B;B;B;A;A;A;C;A;B;C
B;B;A;A;A;B;C;A;B;C
B;B;B;A;A;A;C;A;B;C
C;B;A;A;A;A;B;C;B;B
B;A;A;A;A;C;B;C;B;B
B;A;B;A;A;C;C;A;B;B
B;A;A;A;A;C;B;C;B;B
C;B;A;A;A;A;B;C;B;B
B;B;B;A;A;A;C;A;B;C
B;B;A;A;A;B;C;A;B;C
C;B;B;A;A;B;A;A;C;B
B;C;B;B;A;A;A;A;C;B
C;B;B;A;A;B;A;A;C;B
A;B;C;A;A;B;A;B;B;C
B;C;A;A;B;A;A;B;B;C
C;C;B;A;B;A;B;A;A;B
A;C;C;B;A;B;B;A;A;B
B;C;A;B;A;B;A;C;B;A
A;C;C;B;A;B;B;A;A;B
C;C;B;A;B;A;B;A;A;B
A;C;C;B;A;B;B;A;A;B
B;C;A;B;A;B;A;B;C;A
B;B;C;A;B;A;A;B;C;A
A;B;B;A;B;A;B;A;C;C
B;B;A;B;A;A;B;A;C;C
B;B;C;B;A;A;A;A;C;B
A;B;B;C;B;A;A;A;C;B
A;B;C;C;B;A;A;B;B;A
A;A;B;C;C;B;A;B;B;A
A;B;C;C;B;A;A;B;B;A
A;B;B;C;B;A;A;A;C;B
A;A;B;B;C;B;A;A;C;B
A;B;B;C;B;A;A;A;C;B
B;B;A;C;B;A;A;A;B;C


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]
